# Analyse des perspectives economiques et de robustesse

## Importation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

## Fonctions

In [2]:
def get_OLS(x,y):
    x_with_const = sm.add_constant(x)
    # Modélisation de la régression
    model = sm.OLS(y, x_with_const)
    results = model.fit()
    return results

## Données

In [3]:
SSI = pd.read_csv(r'..\data\SSI.csv',index_col=0)
SSI.columns=['SSI']
SSI

,SSI
Dates,
2006-08-31,-4.573529
2006-09-30,-1.902706
2006-10-31,-0.911145
2006-11-30,-0.087120
2006-12-31,-0.488747
...,...
2023-11-30,0.039909
2023-12-31,-0.387040
2024-01-31,0.338068


In [4]:
SP500 = pd.read_excel(r"..\data\sp500.xlsx",index_col=0)[['PX_LAST']]
SP500_lags=SP500
SP500=SP500.reset_index(drop=False).resample('M', on='Date').mean().pct_change()

C:\Users\collo\AppData\Local\Temp\ipykernel_18136\2770235541.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  SP500=SP500.reset_index(drop=False).resample('M', on='Date').mean().pct_change()


In [5]:
for i in range(1,11):   
    SP500_lags[f"lag_{i}"]= SP500_lags.PX_LAST.pct_change(i)
for i in range(15,21,5):   
    SP500_lags[f"lag_{i}"]= SP500_lags.PX_LAST.pct_change(i)
SP500_lags=SP500_lags.reset_index(drop=False).resample('M', on='Date').sum()

C:\Users\collo\AppData\Local\Temp\ipykernel_18136\3589867855.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  SP500_lags=SP500_lags.reset_index(drop=False).resample('M', on='Date').sum()


## Predictive sans SSI

### Full sample

In [6]:
# Données d'exemple
x = SP500['PX_LAST'][6:168].tolist()
y = SP500['PX_LAST'][7:169].tolist()
# Modélisation de la régression
results = get_OLS(x, y)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.060
Method:                 Least Squares   F-statistic:                     11.27
Date:                Sun, 14 Apr 2024   Prob (F-statistic):           0.000982
Time:                        18:12:36   Log-Likelihood:                 314.18
No. Observations:                 162   AIC:                            -624.4
Df Residuals:                     160   BIC:                            -618.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0049      0.003      1.768      0.0

### Post 2009

In [7]:
# Données d'exemple
x = SP500['PX_LAST'][6:168][30:]
y = SP500['PX_LAST'][6:169][30:].shift(-1).dropna()
x.index = y.index
# Modélisation de la régression
results = get_OLS(x, y)
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                PX_LAST   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     2.522
Date:                Sun, 14 Apr 2024   Prob (F-statistic):              0.115
Time:                        18:12:36   Log-Likelihood:                 274.50
No. Observations:                 132   AIC:                            -545.0
Df Residuals:                     130   BIC:                            -539.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0092      0.003      3.285      0.0

## Predictive avec SSI

### Full sample

In [8]:
# Données d'exemple
x = pd.DataFrame([SP500['PX_LAST'][6:168].tolist(),SSI['SSI'][:162]],index=['SP500',"SSI"]).T
y = SP500['PX_LAST'][7:169].tolist()
# Modélisation de la régression
results = get_OLS(x, y)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.416
Model:                            OLS   Adj. R-squared:                  0.408
Method:                 Least Squares   F-statistic:                     56.58
Date:                Sun, 14 Apr 2024   Prob (F-statistic):           2.77e-19
Time:                        18:12:36   Log-Likelihood:                 352.20
No. Observations:                 162   AIC:                            -698.4
Df Residuals:                     159   BIC:                            -689.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0021      0.002      0.939      0.3

### post - 2009

In [9]:
# Données d'exemple
x = pd.DataFrame([SP500['PX_LAST'][6:168][30:].tolist(),SSI['SSI'][:162][30:]],index=['SP500',"SSI"]).T
y = SP500['PX_LAST'][7:169][30:].tolist()
# Modélisation de la régression
results = get_OLS(x, y)
print(results.summary())
# Données d'exemple
x = SSI['SSI'][:162][30:]
y = SP500['PX_LAST'][6:169][30:].shift(-1).dropna()
x.index = y.index
# Modélisation de la régression
results = get_OLS(x, y)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.302
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     27.88
Date:                Sun, 14 Apr 2024   Prob (F-statistic):           8.65e-11
Time:                        18:12:36   Log-Likelihood:                 296.94
No. Observations:                 132   AIC:                            -587.9
Df Residuals:                     129   BIC:                            -579.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0033      0.003      1.298      0.1

## Horizon


In [10]:
SP500_lags.drop('PX_LAST',inplace=True,axis=1)
SP500_lags

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_15,lag_20
Date,,,,,,,,,,,,
2006-01-31,0.009213,0.018633,0.026953,0.018243,-0.001700,-0.019504,-0.043423,-0.062820,-0.064891,-0.069026,-0.002908,0.000000
2006-02-28,0.000769,0.007813,0.012556,0.023845,0.046017,0.058982,0.077593,0.099587,0.095478,0.093586,-0.005811,-0.027897
2006-03-31,0.011276,0.016190,0.026851,0.031368,0.038482,0.054156,0.065371,0.077493,0.090929,0.113884,0.257204,0.275864
2006-04-30,0.012412,0.019984,0.022232,0.029022,0.034048,0.040431,0.047779,0.051342,0.059281,0.043912,0.012057,0.115994
2006-05-31,-0.030724,-0.068840,-0.087931,-0.109535,-0.147060,-0.188752,-0.226449,-0.258932,-0.293797,-0.304901,-0.264960,-0.208540
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-30,0.086089,0.175188,0.278091,0.375386,0.462706,0.534731,0.609990,0.685789,0.741947,0.788613,0.884177,0.953217
2023-12-31,0.043677,0.093913,0.142940,0.191627,0.234290,0.275762,0.311127,0.359378,0.409369,0.456061,0.710554,0.957720
2024-01-31,0.016260,0.045988,0.076622,0.101123,0.130337,0.155911,0.190902,0.208207,0.229308,0.242571,0.362258,0.524354


In [11]:
# Données d'exemple
for i in range(1,11):
    x = SSI['SSI'][:162].tolist()
    y = SP500_lags[f'lag_{i}'][7:169].tolist()
    # Modélisation de la régression
    results = get_OLS(x, y)
    print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.285
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     63.88
Date:                Sun, 14 Apr 2024   Prob (F-statistic):           2.47e-13
Time:                        18:12:36   Log-Likelihood:                 316.93
No. Observations:                 162   AIC:                            -629.9
Df Residuals:                     160   BIC:                            -623.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0042      0.003      1.553      0.1

In [14]:
from itertools import chain

lags = chain(range(1,11),range(15,21,5))
for i in lags:
    print(i)

1
2
3
4
5
6
7
8
9
10
15
20
